In [1]:
from tensorflow.keras.models import load_model

In [2]:
# load model
path_to_model = 'd:\\CodePratice\\Python\\image_captioning\\flask-backend\\trained_model\\model.keras'
path_to_caption_model = 'd:\\CodePratice\\Python\\image_captioning\\flask-backend\\trained_model\\caption_model.keras'
model = load_model(path_to_model)
caption_model = load_model(path_to_caption_model)


d:\Prog\Python\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
d:\Prog\Python\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
